## Tokenizing .loom single cell RNA-seq data to rank value encoding .dataset format

In [ ]:
from geneformer import TranscriptomeTokenizer
from pathlib import Path

#loom_data_directory = Path("/webdav/MyData/Geneformer2/Genecorpus-30M/data/")
loom_data_directory = Path("/webdav/MyData/Geneformer2/Genecorpus-30M/data/susbet")
print(loom_data_directory.glob("*.loom"))
loom_data_directory.glob("*.loom")
list(loom_data_directory.glob("*.loom"))
output_directory = Path("/webdav/MyData/Geneformer2/Geneformer/data/out")



In [ ]:
import pickle

# 加载 pickle 文件
with open("/webdav/MyData/Geneformer2/Geneformer/geneformer/token_dictionary.pkl", "rb") as f:
    token_dictionary = pickle.load(f)

# 访问字典对象
print(token_dictionary)  # 输出 hello 的标记 ID

In [ ]:
!pip install -U loompy
!pip install pyensembl

In [ ]:
from datasets import load_from_disk
# load evaluation dataset (includes all tissues)
eval_dataset=load_from_disk("/webdav/MyData/Geneformer2/Genecorpus-30M/genecorpus_30M_2048.dataset")
eval_dataset

In [ ]:
from datasets import load_from_disk
eval_datas=load_from_disk("/webdav/MyData/Geneformer2/Geneformer/data/mydata/output_prefix.dataset")
eval_datas

In [ ]:
eval_dataset[2,2]

In [ ]:
import loompy
#ds = loompy.connect("/webdav/MyData/Geneformer2/Genecorpus-30M/data/l5_all.loom")
#ds = loompy.connect("/webdav/MyData/Geneformer2/Genecorpus-30M/data/susbet/1.loom")
ds = loompy.connect("/webdav/MyData/Geneformer2/Geneformer/data/loom12.loom")


ds

In [ ]:
ds[1,:]
ds.ca.keys()
#ds.ra.keys()
layer_names = ds.layers.keys()
# 打印层名称
print(layer_names)
#ds[2, :]   
#ds.shape

In [ ]:
layer_names = ds.layers.keys()
# 打印层名称
print(layer_names)

In [ ]:
ds.ra['Accession']
ds
ds.ra["filter_pass"] = ds.ra["_Valid"]
ds.ca["filter_pass"] 
ds.ra["ensembl_id"] = ds.ra["Accession"]
ds.ca["n_counts"] = ds.ca["_NGenes"]
ds.ca["cell_type"] = ds.ca["Subclass"]
ds.ca["organ_major"] = ds.ca["Tissue"]



In [ ]:
ds.ra["ensembl_id"] = ds.ra["Accession"]


In [ ]:
ds.ca["n_counts"] = ds.ca["_NGenes"]
ds.ca["cell_type"] = ds.ca["Subclass"]
ds.ca["organ_major"] = ds.ca["Tissue"]


In [ ]:
ds.ca["Tissue"].shape
ds.ca["disease"]
ds[1:5,1:5]

arr = ds[1:5000:,1:5000]




In [ ]:
a = ds.ra["ensembl_id", "Gene"] 
a
# Returns a 2D array of shape (n_genes, 2)
b = ds.ca    
b
# # Returns a 2D array of shape (n_cells, 2)



In [ ]:
import loompy
import numpy as np

filename = "/webdav/MyData/Geneformer2/Genecorpus-30M/data/l5_all.loom"
new_filename = "/webdav/MyData/Geneformer2/Genecorpus-30M/data/susbet/1.loom"

# Open the original Loom file
with loompy.connect(filename) as ds:
    # Get the first 100 rows and columns of the matrix
    matrix = ds[:, :6000][:6000, :]
    import numpy as np
    matrix = matrix.astype(np.int64)
    # Get the first 100 elements of the row and column attributes
    col_attrs = { "n_counts": ds.ca["_NGenes"][:6000].astype(np.int64), "cell_type":ds.ca["Subclass"][:6000],"organ_major":ds.ca["Tissue"][:6000],"filter_pass":ds.ca["filter_pass"][:6000],"disease":ds.ca["disease"][:6000]}
    col_attrs
    row_attrs = { "ensembl_id": ds.ra["Accession"][:6000],"filter_pass": ds.ra["filter_pass"][:6000]}
    row_attrs

# Create a new Loom file with the extracted data
loompy.create(new_filename, matrix, row_attrs, col_attrs)

In [ ]:
ds.ca["_NGenes"][:6000]
#ds.ca["filter_pass"][:6000]

print(matrix.dtype)

print(ds.ra["filter_pass"][:6000].dtype)


In [ ]:
ds.ca["AnalysisProject"]

import numpy as np

# 创建一个包含示例数据的 NumPy 数组
arr = ds.ca["cell_type"]

# 使用 unique 函数获取唯一值
unique_values = np.unique(arr)

# 输出唯一值的数量
print(unique_values)





In [ ]:
import numpy as np

#char_array = np.array(['dcm', 'nf', 'hcm'])
ensembl_array = np.random.choice(['dcm', 'nf', 'hcm'], size=(160796,))
#result_array = np.core.defchararray.add(char_array[np.random.choice(3, size=(160796,))], ensembl_array)
ensembl_array
# 转换数据类型为 object
arr = ensembl_array.astype(object)
# 查看转换后的数组
print(arr)
arr
ds.ca["disease"] = arr


In [ ]:
ds.close()

In [ ]:
tk = TranscriptomeTokenizer({"cell_type": "cell_type", "organ_major": "organ_major",'disease':'disease'}, nproc=10)   # Dictionary of custom attributes to be added to the dataset.
tk.tokenize_data(loom_data_directory, output_directory, "output_prefix")

In [ ]:
import pickle
from pathlib import Path

import loompy as lp
import numpy as np
from datasets import Dataset

GENE_MEDIAN_FILE = Path(__file__).parent / "gene_median_dictionary.pkl"
TOKEN_DICTIONARY_FILE = Path(__file__).parent / "token_dictionary.pkl"
GENE_MEDIAN_FILE